In [3]:
import pandas as pd
import numpy as np


In [1]:
!pip install spacy transformers nltk

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB 1.3 MB/s eta 0:00:09
   ---------------------------------------- 0.1/10.4 MB 1.4 MB/s eta 0:00:08
    --------------------------------------- 0.2/10.4 MB 2.0 MB/s eta 0:00:06
    --------------------------------------- 0.3/10.4 MB 1.6 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.4 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.4 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.4 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.4 MB 883.3 kB/s eta 0:00:12
   - -------------------------------------- 0.3/10.4 MB 853.3 kB/s eta 0:00:12
   - -------------------------------------- 0.3/10.4 MB 838.1 kB/s eta 0:00:12
   - -------------------------------------- 0.4/10.4 MB 935.2 kB/s eta 0:00:11
   - -------------------------------------- 0.4/10.4 MB 949.4 kB/s eta 0:00

In [4]:
df=pd.read_csv("CNN_daily_email dataset/cnn_dailymail/CNN_daily_emails_train.csv")
df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [6]:
df.shape

(287113, 3)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [11]:
df.describe()

,id,article,highlights
count,287113,287113,287113
unique,287113,284005,282197
top,0001d1afc246a7964130f43ae940af6bc6c57f01,(CNN) -- Dubai could lose its place on the Wom...,This page includes the show Transcript and the...
freq,1,3,83


In [ ]:
import nltk
import spacy
from heapq import nlargest
from transformers import pipeline
import re
from nltk.tokenize import sent_tokenize

In [ ]:
def extractive_summary(text, num_sentences=3):
    # Tokenize into sentences
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    
    if len(sentences) <= num_sentences:
        return ' '.join(sentences)

    # TF-IDF based sentence similarity
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    scores = np.sum(X.toarray(), axis=1)

    # Get top N scored sentence indices
    top_n_idx = scores.argsort()[-num_sentences:][::-1]

    # Sort them in the order they appear in the text
    top_n_idx_sorted = sorted(top_n_idx)
    
    summary = ' '.join([sentences[i] for i in top_n_idx_sorted])
    return summary
# Apply to a small sample (for performance)
df['extractive_summary'] = df['clean_article'].head(100).apply(lambda x: extractive_summary(x, num_sentences=3))

# View sample
print(df[['clean_article', 'extractive_summary']].head())


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
def abstractive_summary(text):
    summary = summarizer(text, max_length=109, min_length=30, do_sample=False)
    return summary[0]['summary_text']

In [14]:
import pandas as pd
import re


# Function to clean article and summary
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)              # remove excessive whitespace
    text = re.sub(r'\[[^]]*\]', '', text)         # remove content in square brackets
    text = re.sub(r'\(.*?\)', '', text)           # remove content in parentheses
    text = re.sub(r'\n', ' ', text)               # remove newline characters
    return text.strip()

# Apply cleaning to both article and highlights
df['clean_article'] = df['article'].apply(clean_text)
df['clean_summary'] = df['highlights'].apply(clean_text)


print(df[['clean_article', 'clean_summary']].head())


                                       clean_article  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  -- Ralph Mata was an internal affairs lieutena...   
2  A drunk driver who killed a young woman in a h...   
3  -- With a breezy sweep of his pen President Vl...   
4  Fleetwood are the only team still to have a 10...   

                                       clean_summary  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...  
